In [15]:
from entsoe import EntsoePandasClient
import pandas as pd
import math

client = EntsoePandasClient(api_key="b18dfce9-f1e3-4d07-822f-4abd1438e602")

In [4]:
start = pd.Timestamp('20220901', tz='Europe/Amsterdam')
end = pd.Timestamp('20230331', tz='Europe/Amsterdam')

start_cap = pd.Timestamp('20200101', tz='Europe/Amsterdam')
end_cap = pd.Timestamp('20230101', tz='Europe/Amsterdam')

countries = ['AT', 'BE', 'CZ', 'DE_LU', 'FR', 'HR', 'HU', 'NL', 'PL', 'RO', 'SI', 'SK']
non_fbmc_countries = ['CH', 'GB', 'ES', 'IT_NORD']

hubs = ['ALBE', 'ALDE', 'AT', 'BE', 'CZ', 'DE_LU', 'FR', 'HR', 'HU', 'NL', 'PL', 'RO', 'SI', 'SK']

borders = [
    ['FR', 'CH'],
    ['FR', 'ES'],
    ['FR', 'GB'],
    ['FR', 'IT_NORD'],
    ['AT', 'IT_NORD'],
    ['AT', 'CH'],
    ['BE', 'GB'],
    ['NL', 'GB'],
    ['SI', 'IT_NORD'],
    ['DE_LU', 'CH'],
    ['IT_NORD', 'CH'],
]

gen_types = [
    'Biomass', 'Fossil Brown coal/Lignite', 'Fossil Coal-derived gas', 'Fossil Gas', 'Fossil Hard coal', 
    'Fossil Oil', 'Fossil Oil shale', 'Fossil Peat', 'Geothermal', 'Hydro Pumped Storage', 
    'Hydro Run-of-river and poundage', 'Hydro Water Reservoir', 'Marine', 'Nuclear', 'Other',
    'Other renewable', 'Solar', 'Waste', 'Wind Offshore', 'Wind Onshore'
]

standardized_gen_types = [
    'biomass', 'brown_coal', 'coal_gas', 'natural_gas', 'hard_coal', 'oil', 'oil_shale', 'peat', 'geothermal', 
    'hydro_pumped_storage', 'hydro_run_of_river', 'hydro_reservoir', 'marine', 'nuclear', 'other', 'other_renewable',
    'solar', 'waste', 'wind_offshore', 'wind_onshore'
]

reduced_gen_types = [
    'biomass', 'brown_coal', 'coal_gas', 'natural_gas', 'hard_coal', 'oil', 'hydro', 
    'nuclear', 'waste', 'other', 'solar', 'wind_onshore', 'wind_offshore', 'other_renewable'
]

In [5]:
def standardize_generation_data(df_gen):
    for gen_type in gen_types:
        if gen_type not in df_gen:
            df_gen[gen_type] = 0.0

    for gen_type in gen_types:
        if (gen_type, 'Actual Consumption') in df_gen:
            df_gen = df_gen.drop(columns=[(gen_type, 'Actual Consumption')], axis=1)

    df_gen = df_gen[gen_types]
    df_gen.columns = standardized_gen_types

    df_gen = df_gen.tz_localize(None)

    df_gen = df_gen.resample("H").agg(["mean"])
    df_gen.columns = standardized_gen_types
    
    return df_gen

## Day-ahead prices: FBMC

In [6]:
df_da_prices = pd.DataFrame()
for country_code in countries:
    df_da = client.query_day_ahead_prices(country_code, start=start, end=end)
    print(country_code)
    
    df_da = df_da.to_frame()
    df_da = df_da.rename(columns={
        0: country_code, 
    })

    df_da = df_da.tz_localize(None)
    df_da = df_da.resample("H").agg(["mean"])
    df_da.columns = [country_code]
    
    if df_da_prices.empty:
        df_da_prices = df_da.copy()
    else:
        df_da_prices[country_code] = df_da[country_code]

df_da_prices['ALBE'] = df_da_prices['BE']
df_da_prices['ALDE'] = df_da_prices['DE_LU']
df_da_prices = df_da_prices[hubs]

df_da_prices.to_excel("./data/day_ahead_prices.xlsx")

AT
BE
CZ
DE_LU
FR
HR
HU
NL
PL
RO
SI
SK


## Day-ahead prices: Non-FBMC

In [8]:
df_da_prices = pd.DataFrame()
for country_code in [x for x in non_fbmc_countries if x != "GB"]:
    df_da = client.query_day_ahead_prices(country_code, start=start, end=end)
    print(country_code)
    
    df_da = df_da.to_frame()
    df_da = df_da.rename(columns={
        0: country_code, 
    })

    df_da = df_da.tz_localize(None)
    df_da = df_da.resample("H").agg(["mean"])
    df_da.columns = [country_code]
    
    if df_da_prices.empty:
        df_da_prices = df_da.copy()
    else:
        df_da_prices[country_code] = df_da[country_code]

df_da_prices.to_excel("./data/day_ahead_prices_non_fbmc.xlsx")

# UK: DAY-AHEAD PRICES
df_uk_settlements = pd.read_excel("./data/uk_settlements.xlsx", index_col=0)
df_uk_settlements = df_uk_settlements[["EPEX UK EUR"]]
df_uk_settlements = df_uk_settlements.tz_localize(None)
df_uk_settlements = df_uk_settlements.resample("H").agg(["mean"])
df_uk_settlements.columns = ['epex']

df_da_prices['GB'] = df_uk_settlements['epex']
df_da_prices = df_da_prices[non_fbmc_countries]

df_da_prices.to_excel("./data/day_ahead_prices_non_fbmc.xlsx")

CH
ES
IT_NORD


## Demand: FBMC

In [9]:
df_demand = pd.DataFrame()
for country_code in countries:
    df_da = client.query_load_forecast(country_code, start=start, end=end)
    print(country_code)

    df_da = df_da.rename(columns={
        'Forecasted Load': country_code, 
    })
    
    df_da = df_da.tz_localize(None)
    df_da = df_da.resample("H").agg(["mean"])
    df_da.columns = [country_code]
    
    if df_demand.empty:
        df_demand = df_da.copy()
    else:
        df_demand[country_code] = df_da[country_code]
        
df_demand['ALBE'] = 0
df_demand['ALDE'] = 0
df_demand = df_demand[hubs]
        
df_demand.to_excel("./data/demand.xlsx")

## Demand: Non-FBMC

In [11]:
df_demand = pd.DataFrame()
for country_code in [x for x in non_fbmc_countries if x != "GB"]: # NO GB
    df_da = client.query_load_forecast(country_code, start=start, end=end)
    print(country_code)
    
    df_da = df_da.rename(columns={
        'Forecasted Load': country_code, 
    })
    
    df_da = df_da.tz_localize(None)
    df_da = df_da.resample("H").agg(["mean"])
    df_da.columns = [country_code]
    
    if df_demand.empty:
        df_demand = df_da.copy()
    else:
        df_demand[country_code] = df_da[country_code]
        
df_demand.to_excel("./data/demand_non_fbmc.xlsx")

# UK
df_uk_gridwatch = pd.read_csv("./data/uk_gridwatch.csv", index_col=1)
df_uk_gridwatch.index = pd.to_datetime(df_uk_gridwatch.index)
df_uk_gridwatch = df_uk_gridwatch[[
    ' demand', ' coal', ' nuclear', ' ccgt', ' pumped', ' hydro', ' biomass', ' oil', ' ocgt', ' wind', ' solar' 
]]
df_uk_gridwatch = df_uk_gridwatch.tz_localize(None)
df_uk_gridwatch = df_uk_gridwatch.resample("H").agg(["mean"])
df_uk_gridwatch.columns = [
    'demand', 'coal', 'nuclear', 'ccgt', 'pumped', 'hydro', 'biomass', 'oil', 'ocgt', 'wind', 'solar'  
]

df_demand['GB'] = df_uk_gridwatch['demand']
df_demand = df_demand[non_fbmc_countries]
df_demand.to_excel("./data/demand_non_fbmc.xlsx")

CH
ES
IT_NORD


## Generation: FBMC

In [12]:
with pd.ExcelWriter("./data/generation.xlsx") as writer:  
    for country_code in countries:
        df_gen = client.query_generation(country_code, start=start,end=end)
        print(country_code)

        df_gen = standardize_generation_data(df_gen)
        df_gen.loc[:, 'hydro'] = df_gen['hydro_pumped_storage'] + df_gen['hydro_run_of_river'] + df_gen['hydro_reservoir']
        df_gen = df_gen[reduced_gen_types]
        
        df_gen.to_excel(writer, sheet_name=country_code) 

AT
BE
CZ
DE_LU
FR
HR
HU
NL
PL
RO
SI
SK


## Generation: Non-FBMC

In [13]:
with pd.ExcelWriter("./data/generation_non_fbmc.xlsx") as writer:  
    for country_code in [x for x in non_fbmc_countries if x != "GB"]: # NO GB
        df_gen = client.query_generation(country_code, start=start,end=end)
        print(country_code)
        
        df_gen = standardize_generation_data(df_gen)
        df_gen.loc[:, 'hydro'] = df_gen['hydro_pumped_storage'] + df_gen['hydro_run_of_river'] + df_gen['hydro_reservoir']
        df_gen = df_gen[reduced_gen_types]
   
        df_gen.to_excel(writer, sheet_name=country_code) 
    
# UK
df_gen = pd.read_excel("./optimisation/data/generation_non_fbmc.xlsx", sheet_name='CH', index_col=0)
df_gen_uk = pd.DataFrame()
df_gen_uk.index = df_gen.index

df_gen_uk['biomass'] = df_uk_gridwatch['biomass']
df_gen_uk['brown_coal'] = 0
df_gen_uk['coal_gas'] = 0
df_gen_uk['natural_gas'] = df_uk_gridwatch['ccgt'] + df_uk_gridwatch['ocgt']
df_gen_uk['hard_coal'] = df_uk_gridwatch['coal']
df_gen_uk['oil'] = df_uk_gridwatch['oil']
df_gen_uk['hydro'] = df_uk_gridwatch['pumped'] + df_uk_gridwatch['hydro']
df_gen_uk['nuclear'] = df_uk_gridwatch['nuclear']
df_gen_uk['waste'] = 0
df_gen_uk['other'] = 0

df_gen_uk['solar'] = df_uk_gridwatch['solar']
df_gen_uk['wind_onshore'] = 0
df_gen_uk['wind_offshore'] = df_uk_gridwatch['wind']
df_gen_uk['other_renewable'] = 0

df_gen_uk = df_gen_uk[reduced_gen_types]
df_gen_uk.to_excel("./data/generation_uk_non_fbmc.xlsx") # COPY THIS TO generation_non_fbmc.xlsx UK TAB!

CH
ES
IT_NORD


## Installed capacities: FBMC

In [16]:
df_capacities = pd.DataFrame()
for country_code in countries:
    df_cap = client.query_installed_generation_capacity(country_code, start=start_cap, end=end_cap)
    print(country_code)
    
    for gen_type in gen_types:
        if gen_type not in df_cap:
            df_cap[gen_type] = 0.0

    df_cap = df_cap[gen_types]
    df_cap.columns = standardized_gen_types
    
    df_cap = df_cap.iloc[-1].to_frame()
    df_cap = df_cap.rename(columns={
        df_cap.columns[0]: country_code
    })
    
    if df_capacities.empty:
        df_capacities = df_cap.copy()
    else:
        df_capacities[country_code] = df_cap[country_code]
        
df_capacities = df_capacities.drop([df_capacities.index[6], df_capacities.index[7], df_capacities.index[8], df_capacities.index[12]])
df_capacities = df_capacities.rename(index={'hydro_pumped_storage': 'hydro'})
    
for column in df_capacities:
    df_capacities.loc['hydro', column] = df_capacities.loc['hydro', column] + df_capacities.loc['hydro_run_of_river', column] + df_capacities.loc['hydro_reservoir', column]

df_capacities = df_capacities.drop([df_capacities.index[7], df_capacities.index[8]])

df_capacities['ALBE'] = 0
df_capacities['ALDE'] = 0
df_capacities = df_capacities[hubs]

df_capacities = df_capacities.reindex(reduced_gen_types)

# capacity correction for larger generation values
for zone in countries:
    for gen_type in reduced_gen_types:
        df_gen = pd.read_excel("./optimisation/data/training/generation.xlsx", sheet_name=zone, index_col=0)
        if not df_gen[df_gen[gen_type] > df_capacities[zone].loc[gen_type]].empty:
            df_excess_gen = df_gen[df_gen[gen_type] > df_capacities[zone].loc[gen_type]]
            print(zone, ', ', gen_type, ', max: ', df_capacities[zone].loc[gen_type], ', gen: ', math.ceil(df_excess_gen[gen_type].max()))
            df_capacities.loc[gen_type, zone] = math.ceil(df_excess_gen[gen_type].max())
        
df_capacities.to_excel("./data/installed_capacities.xlsx")

AT
BE
CZ
DE_LU
FR
HR
HU
NL
PL
RO
SI
SK
BE ,  other , max:  19.0 , gen:  2049
CZ ,  natural_gas , max:  1240.0 , gen:  1279
CZ ,  oil , max:  0.0 , gen:  35
CZ ,  other , max:  0.0 , gen:  123
DE_LU ,  nuclear , max:  4056.0 , gen:  4091
FR ,  hard_coal , max:  1816.0 , gen:  1840
HR ,  waste , max:  6.0 , gen:  9
HU ,  hard_coal , max:  42.0 , gen:  43
HU ,  nuclear , max:  1916.0 , gen:  1960
HU ,  other , max:  0.0 , gen:  96
NL ,  nuclear , max:  486.0 , gen:  490
NL ,  other , max:  1.0 , gen:  5100
PL ,  solar , max:  6664.0 , gen:  6711
RO ,  nuclear , max:  1300.0 , gen:  1420
SI ,  oil , max:  58.0 , gen:  60
SI ,  nuclear , max:  696.0 , gen:  702
SI ,  wind_onshore , max:  2.0 , gen:  3
SK ,  hard_coal , max:  1.0 , gen:  165
SK ,  nuclear , max:  2003.0 , gen:  2009


## Installed capacities: Non-FBMC

In [17]:
df_capacities = pd.DataFrame()
for country_code in [x for x in non_fbmc_countries if x != "IT_NORD"]: # NO IT_NORD
    df_cap = client.query_installed_generation_capacity(country_code, start=start_cap, end=end_cap)
    print(country_code)
    
    for gen_type in gen_types:
        if gen_type not in df_cap:
            df_cap[gen_type] = 0.0

    df_cap = df_cap[gen_types]
    df_cap.columns = standardized_gen_types
    
    df_cap = df_cap.iloc[-1].to_frame()
    df_cap = df_cap.rename(columns={
        df_cap.columns[0]: country_code
    })
    
    if df_capacities.empty:
        df_capacities = df_cap.copy()
    else:
        df_capacities[country_code] = df_cap[country_code]
         
df_capacities = df_capacities.drop([df_capacities.index[6], df_capacities.index[7], df_capacities.index[8], df_capacities.index[12]])
df_capacities = df_capacities.rename(index={'hydro_pumped_storage': 'hydro'})
    
for column in df_capacities:
    df_capacities.loc['hydro', column] = df_capacities.loc['hydro', column] + df_capacities.loc['hydro_run_of_river', column] + df_capacities.loc['hydro_reservoir', column]

df_capacities = df_capacities.drop([df_capacities.index[7], df_capacities.index[8]])

df_capacities = df_capacities.reindex(reduced_gen_types)

df_capacities.to_excel("./data/installed_capacities_non_fbmc.xlsx")

# Update IT North capacities manually

CH
GB
ES


## Generation outages: FBMC

In [18]:
df_gen = pd.read_excel('./data/generation.xlsx', sheet_name='AT', index_col=0)

with pd.ExcelWriter("./data/generation_outages.xlsx") as writer:
    for country_code in countries:
        print(country_code)
        
        try:
            df_out_gen = client.query_unavailability_of_generation_units(country_code, start=start, end=end)
            df_out_gen = df_out_gen.tz_localize(None)
            df_out_gen['end'] = df_out_gen['end'].dt.tz_localize(None)
            df_out_gen['start'] = df_out_gen['start'].dt.tz_localize(None)

            df_outages_zone = pd.DataFrame()

            for gen_date in df_gen.index:
                df_out_gen_test = df_out_gen[(df_out_gen.start <= gen_date) & (df_out_gen.end >= gen_date) & (df_out_gen.docstatus != 'Cancelled')]

                for gen_type in standardized_gen_types:
                        df_outages_zone.loc[gen_date, gen_type] = 0

                for index, row in df_out_gen_test.iterrows():
                    df_outages_zone.loc[gen_date, standardized_gen_types[gen_types.index(row['plant_type'])]] += float(row['nominal_power']) - float(row['avail_qty'])
                    df_outages_zone.loc[gen_date, 'hydro'] = df_outages_zone.loc[gen_date, 'hydro_pumped_storage'] + df_outages_zone.loc[gen_date, 'hydro_run_of_river'] + df_outages_zone.loc[gen_date, 'hydro_reservoir']

            df_outages_zone = df_outages_zone[reduced_gen_types]
            df_outages_zone.to_excel(writer, sheet_name=country_code)
        except:
            print("An exception occurred")
            df_outages_zone = pd.DataFrame()
            for gen_date in df_gen.index:
                for gen_type in reduced_gen_types:
                    df_outages_zone.loc[gen_date, gen_type] = 0
            df_outages_zone.to_excel(writer, sheet_name=country_code)

AT
BE
CZ
DE_LU
FR
HR
An exception occurred
HU
NL
PL
RO
SI
SK


## Generation outages: Non-FBMC

In [19]:
with pd.ExcelWriter("./data/generation_outages_non_fbmc.xlsx") as writer:
    for country_code in non_fbmc_countries:
        print(country_code)
        
        try:
            df_out_gen = client.query_unavailability_of_generation_units(country_code, start=start, end=end)
            df_out_gen = df_out_gen.tz_localize(None)
            df_out_gen['end'] = df_out_gen['end'].dt.tz_localize(None)
            df_out_gen['start'] = df_out_gen['start'].dt.tz_localize(None)

            df_outages_zone = pd.DataFrame()

            for gen_date in df_gen.index:
                df_out_gen_test = df_out_gen[(df_out_gen.start <= gen_date) & (df_out_gen.end >= gen_date) & (df_out_gen.docstatus != 'Cancelled')]

                for gen_type in standardized_gen_types:
                        df_outages_zone.loc[gen_date, gen_type] = 0

                for index, row in df_out_gen_test.iterrows():
                    df_outages_zone.loc[gen_date, standardized_gen_types[gen_types.index(row['plant_type'])]] += float(row['nominal_power']) - float(row['avail_qty'])
                    df_outages_zone.loc[gen_date, 'hydro'] = df_outages_zone.loc[gen_date, 'hydro_pumped_storage'] + df_outages_zone.loc[gen_date, 'hydro_run_of_river'] + df_outages_zone.loc[gen_date, 'hydro_reservoir']

            df_outages_zone = df_outages_zone[reduced_gen_types]
            df_outages_zone.to_excel(writer, sheet_name=country_code)
        except:
            print("An exception occurred")
            df_outages_zone = pd.DataFrame()
            for gen_date in df_gen.index:
                for gen_type in reduced_gen_types:
                    df_outages_zone.loc[gen_date, gen_type] = 0
            df_outages_zone.to_excel(writer, sheet_name=country_code)

CH
GB
ES
IT_NORD


## Renewable generation forecast: FBMC

In [20]:
df_ren_gen = pd.DataFrame()

for country_code in countries:
    df_gen = client.query_wind_and_solar_forecast(country_code, start=start,end=end)
    if 'Solar' not in df_gen:
        df_gen['Solar'] = 0
    if 'Wind Onshore' not in df_gen:
        df_gen['Wind Onshore'] = 0
    if 'Wind Offshore' not in df_gen:
        df_gen['Wind Offshore'] = 0

    df_gen = df_gen.tz_localize(None)
    df_gen = df_gen.resample("H").agg(["mean"])

    df_ren_gen[country_code] = df_gen['Solar'] + df_gen['Wind Onshore'] + df_gen['Wind Offshore']
    
df_ren_gen['ALBE'] = 0
df_ren_gen['ALDE'] = 0

df_ren_gen = df_ren_gen[hubs]

df_ren_gen.to_excel("./data/renewable_generation.xlsx")

## Renewable generation forecast: Non-FBMC

In [21]:
df_ren_gen = pd.DataFrame()

for country_code in [x for x in non_fbmc_countries if x != "GB"]: # NO GB
    df_gen = client.query_wind_and_solar_forecast(country_code, start=start,end=end)
    print(country_code)
    
    if 'Solar' not in df_gen:
        df_gen['Solar'] = 0
    if 'Wind Onshore' not in df_gen:
        df_gen['Wind Onshore'] = 0
    if 'Wind Offshore' not in df_gen:
        df_gen['Wind Offshore'] = 0

    df_gen = df_gen.tz_localize(None)
    df_gen = df_gen.resample("H").agg(["mean"])

    df_ren_gen[country_code] = df_gen['Solar'] + df_gen['Wind Onshore'] + df_gen['Wind Offshore']
    
df_ren_gen.to_excel("./data/renewable_generation_non_fbmc.xlsx")

# UK: RENEWABLE GENERATION (FORECAST)
df_ren_gen['GB'] = df_uk_gridwatch['wind'] + df_uk_gridwatch['solar']
df_ren_gen = df_ren_gen[non_fbmc_countries]
df_ren_gen.to_excel("./data/renewable_generation_non_fbmc.xlsx")

CH
ES
IT_NORD


## ATC

In [22]:
df_atc = pd.DataFrame()

for border in borders:
    atc_1 = client.query_net_transfer_capacity_dayahead(border[0], border[1], start=start, end=end)
    atc_2 = client.query_net_transfer_capacity_dayahead(border[1], border[0], start=start, end=end)
    
    atc_1 = atc_1.tz_localize(None)
    atc_2 = atc_2.tz_localize(None)
    
    atc_1 = atc_1[~atc_1.index.duplicated()]
    atc_2 = atc_2[~atc_2.index.duplicated()]
    
    df_atc[border[0] + '_' + border[1]] = atc_1
    df_atc[border[1] + '_' + border[0]] = atc_2
    
df_atc.to_excel("./data/atc.xlsx")